# 蔬菜价格预测模型

2018年4月25日

在这个项目中，我们将使用成都市蔬菜价格数据建立一个有关最高价和最低价的预测模型。探索多个常用的监督学习算法并找出其中最优的方案。

## 1. 数据准备

### 1.1 导入数据

首先从CSV文件中导入数据，计算每一条数据前3天和前9天价格平均值，然后将数据拆分为特征和目标两个部分。

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
#读取成都的蔬菜价格数据
data = pd.read_csv('./vegetable_prices.csv')
#读取重庆的蔬菜价格数据
#data = pd.read_csv('./vegetable_prices_Chongqing.csv')
# calculating average prices 3 or 9 days before
data['h_3'] = data['h_price'].shift(1).rolling(window=3).mean()
data['h_9'] = data['h_price'].shift(1).rolling(window=9).mean()
data['l_3'] = data['l_price'].shift(1).rolling(window=3).mean()
data['l_9'] = data['l_price'].shift(1).rolling(window=9).mean()
data = data.dropna()

#通过索引来看，这里的编码是没问题的
# data_encoded = get_dummies(data, columns=['v_name','area','v_market'])
# data_encoded.to_csv("data_encoded.csv", sep=',', index=True, encoding='utf-8')
# data.to_csv("data_1.csv", sep=',', index=True, encoding='utf-8')

prices = data[['h_price', 'l_price']]
features = data.drop(['l_price', 'h_price', 'v_price'], axis=1)
print('chengdu vegetable dataset has {} data points with {} variables each'.format(*data.shape))

chengdu vegetable dataset has 7371 data points with 22 variables each


### 1.2 分析数据

数据包含如下22个变量：

In [13]:
data.head()

,v_name,v_price,h_price,l_price,v_market,area,source,updateTime,insertTime,yWendu,...,fengli,fengxiang,aqi,aqiLevel,aqiInfo,cpi,h_3,h_9,l_3,l_9
9,生姜,7.2,8.0,6.6,四川成都龙泉聚和(国际)果蔬菜交易中心,cd,vegnet.com.cn,2016-03-01,2018-04-19,6,...,微风,南风,137,3,轻度污染,102.3,3.666667,3.377778,2.900000,2.666667
10,大蒜,8.4,8.8,7.8,四川成都龙泉聚和(国际)果蔬菜交易中心,cd,vegnet.com.cn,2016-03-01,2018-04-19,6,...,微风,南风,137,3,轻度污染,102.3,5.600000,4.066667,4.533333,3.266667
11,芹菜,4.8,5.2,4.5,四川成都龙泉聚和(国际)果蔬菜交易中心,cd,vegnet.com.cn,2016-03-01,2018-04-19,6,...,微风,南风,137,3,轻度污染,102.3,7.600000,4.855556,6.466667,4.000000
12,莴笋,2.4,3.0,2.1,四川成都龙泉聚和(国际)果蔬菜交易中心,cd,vegnet.com.cn,2016-03-01,2018-04-19,6,...,微风,南风,137,3,轻度污染,102.3,7.333333,4.833333,6.300000,4.033333
13,蒜薹,9.5,10.0,8.0,四川成都龙泉聚和(国际)果蔬菜交易中心,cd,vegnet.com.cn,2016-03-01,2018-04-19,6,...,微风,南风,137,3,轻度污染,102.3,5.666667,4.555556,4.800000,3.733333


In [14]:
data.describe()

,v_price,h_price,l_price,yWendu,bWendu,aqi,aqiLevel,cpi,h_3,h_9,l_3,l_9
count,7371.000000,7371.000000,7371.000000,7371.000000,7371.000000,7371.000000,7371.000000,7371.000000,7371.000000,7371.000000,7371.000000,7371.000000
mean,4.278974,4.882214,3.843664,12.125085,19.888211,90.249084,2.277574,101.695387,4.881142,4.880273,3.843135,3.842783
std,2.945466,3.311870,2.756567,7.613284,7.914107,47.855732,0.961405,0.563581,1.998666,1.037054,1.626428,0.746328
min,0.800000,0.900000,0.700000,-4.000000,4.000000,25.000000,1.000000,100.800000,1.166667,2.100000,0.900000,1.666667
25%,2.000000,2.300000,1.800000,6.000000,13.000000,54.000000,2.000000,101.300000,3.333333,4.277778,2.533333,3.422222
50%,3.600000,4.000000,3.200000,12.000000,19.000000,79.000000,2.000000,101.400000,4.500000,4.800000,3.533333,3.888889
75%,5.000000,6.000000,4.500000,19.000000,27.000000,117.000000,3.000000,102.300000,6.133333,5.511111,4.700000,4.300000
max,16.500000,20.000000,16.000000,26.000000,36.000000,318.000000,6.000000,102.600000,12.333333,9.700000,8.366667,6.811111


以上是对原始数据集中的几个数值型变量进行的基本统计分析，可以看到均值，标准差，中位数和最值等基本描述统计量。

### 1.3 数据分割与重排
接下来，先将数据集中的蔬菜名称转换为独热编码（One Hot Encoding），然后分成训练和测试两个子集并打乱数据顺序，消除数据集中由于顺序产生的偏差，分割比例为80%数据用于训练，20%用于测试。

In [44]:
from sklearn.model_selection import train_test_split
from pandas import get_dummies

features_encoded = get_dummies(features, columns=['v_name', 'tianqi','area','v_market','aqiInfo'])
X_train, X_test, y_train, y_test = train_test_split(features_encoded, prices, test_size=0.2, random_state=42)

print('dataset train: {} {}, test: {} {}'.format(
    X_train.shape, y_train.shape, X_test.shape, y_test.shape))
features_encoded.head()


dataset train: (5896, 83) (5896, 2), test: (1475, 83) (1475, 2)


,source,updateTime,insertTime,yWendu,bWendu,fengli,fengxiang,aqi,aqiLevel,cpi,...,tianqi_阵雨~小雨,tianqi_阵雨~阴,area_cd,v_market_四川成都龙泉聚和(国际)果蔬菜交易中心,aqiInfo_严重污染,aqiInfo_中度污染,aqiInfo_优,aqiInfo_良,aqiInfo_轻度污染,aqiInfo_重度污染
9,vegnet.com.cn,2016-03-01,2018-04-19,6,19,微风,南风,137,3,102.3,...,0,0,1,1,0,0,0,0,1,0
10,vegnet.com.cn,2016-03-01,2018-04-19,6,19,微风,南风,137,3,102.3,...,0,0,1,1,0,0,0,0,1,0
11,vegnet.com.cn,2016-03-01,2018-04-19,6,19,微风,南风,137,3,102.3,...,0,0,1,1,0,0,0,0,1,0
12,vegnet.com.cn,2016-03-01,2018-04-19,6,19,微风,南风,137,3,102.3,...,0,0,1,1,0,0,0,0,1,0
13,vegnet.com.cn,2016-03-01,2018-04-19,6,19,微风,南风,137,3,102.3,...,0,0,1,1,0,0,0,0,1,0


独热编码后，变量增加到了41个，其中蔬菜名称变成了一系列数值型数据。

## 2. 训练模型

### 2.1 线性回归模型

根据前面对数据进行的探索性分析，我们得到如下结论：

1. 蔬菜最高价与“最低价”、“白天温度”、“夜晚温度”、“前3天最高价均值”、“前9天最高价均值”、“前3天最低价均值”、“前9天最低价均值”具有相关关系；
2. 蔬菜最低价与“最高价”、“前3天最高价均值”、“前9天最高价均值”、“前3天最低价均值”、“前9天最低价均值”具有相关关系；

因此首先想到的是通过多元线性回归建立预测模型。首先需要在分割好的数据集中删掉不相关的变量，计算需要的变量。

In [45]:
X_train_lm = X_train.drop([
    'source', 
    'updateTime', 
    'insertTime', 
    'fengli', 
    'fengxiang', 
    'aqiLevel', 
    'cpi'
], axis=1)

X_test_lm = X_test.drop([
    'source', 
    'updateTime', 
    'insertTime', 
    'fengli', 
    'fengxiang', 
    'aqiLevel', 
    'cpi',
], axis=1)

X_train_lm.head()

,yWendu,bWendu,aqi,h_3,h_9,l_3,l_9,v_name_冬瓜,v_name_南瓜,v_name_土豆,...,tianqi_阵雨~小雨,tianqi_阵雨~阴,area_cd,v_market_四川成都龙泉聚和(国际)果蔬菜交易中心,aqiInfo_严重污染,aqiInfo_中度污染,aqiInfo_优,aqiInfo_良,aqiInfo_轻度污染,aqiInfo_重度污染
6241,0,12,118,3.666667,4.611111,2.166667,2.600000,0,0,0,...,0,0,1,1,0,0,0,0,1,0
2532,14,23,85,2.333333,2.866667,1.933333,2.400000,0,0,0,...,0,0,1,1,0,0,0,1,0,0
5247,14,20,151,4.766667,5.988889,3.600000,4.288889,0,1,0,...,0,0,1,1,0,1,0,0,0,0
5727,8,13,118,4.166667,5.355556,3.200000,4.177778,0,0,0,...,0,0,1,1,0,0,0,0,1,0
3299,8,13,248,2.100000,3.800000,1.700000,3.088889,0,0,0,...,0,0,1,1,0,0,0,0,0,1


下面将使用网格搜索和K折交叉验证的方法，基于线性回归模型建立对价格的预测。

In [46]:
from sklearn.metrics import r2_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

def r2_score_metric(y_true, y_pred):
    score = r2_score(y_true, y_pred)
    return score

def fit_model(X, y, regressor, params):
    cross_validator = KFold(n_splits=5, shuffle=True, random_state=42)
    scoring_func = make_scorer(r2_score_metric)
    grid = GridSearchCV(estimator=regressor, param_grid=params, scoring=scoring_func, cv=cross_validator)
    grid = grid.fit(X, y)
    return grid.best_estimator_

linear_regressor = fit_model(X_train_lm, y_train, LinearRegression(), {
        'fit_intercept': [True, False],
        'normalize': [True, False],
    })

print('paramter for the optimal model {}'.format(linear_regressor.get_params()))

paramter for the optimal model {'copy_X': True, 'normalize': False, 'n_jobs': 1, 'fit_intercept': True}


In [47]:
y_pred = linear_regressor.predict(X_test_lm)
r2 = r2_score_metric(y_test, y_pred)
print('optimal model has R^2 score {:,.2f} on test data'.format(r2))

optimal model has R^2 score 0.88 on test data


In [49]:
result = linear_regressor.predict(features_encoded.drop([
    'source', 
    'updateTime', 
    'insertTime', 
    'fengli', 
    'fengxiang', 
    'aqiLevel', 
    'cpi',
], axis=1))

# result = pd.DataFrame(result)
# result.to_csv("out_1.csv", sep=',', index=True, encoding='utf-8')

print result
result = {"h_price_pred": result[:,0],
          "l_price_pred": result[:,1]}
result = pd.DataFrame(result)
data1 = data.reset_index(drop=True)
data1.head()
result = data1.join(result)
result.head()
result.to_csv("out_chengdu_Line.csv", sep=',', index=True, encoding='utf-8')
#result.to_csv("out_chongqing.csv", sep=',', index=True, encoding='utf-8')

[[ 6.88760455  5.64713814]
 [13.0682888   9.93114224]
 [ 4.70385712  3.86436065]
 ...
 [ 7.78730572  5.66442468]
 [13.88268899  9.95311369]
 [ 5.69496039  3.75632918]]


### 2.2 决策树模型

In [23]:
features_encoded = get_dummies(features, columns=['v_name', 'tianqi','area','v_market','aqiInfo'])
X_train, X_test, y_train, y_test = train_test_split(features_encoded, prices, test_size=0.2, random_state=42)

X_train = X_train.drop([
    'source', 
    'updateTime', 
    'insertTime', 
    'fengli', 
    'fengxiang', 
    'aqiLevel', 
    'cpi',
], axis=1)

X_test = X_test.drop([
    'source', 
    'updateTime', 
    'insertTime', 
    'fengli', 
    'fengxiang', 
    'aqiLevel', 
    'cpi',
], axis=1)

print('dataset train: {} {}, test: {} {}'.format(
    X_train.shape, y_train.shape, X_test.shape, y_test.shape))

dataset train: (5896, 76) (5896, 2), test: (1475, 76) (1475, 2)


In [24]:
from sklearn.tree import DecisionTreeRegressor

dt_regressor = fit_model(X_train, y_train, DecisionTreeRegressor(), {
        'max_depth': range(1, 11),
    })

print('paramter for the optimal model {}'.format(dt_regressor.get_params()))

paramter for the optimal model {'presort': False, 'splitter': 'best', 'min_impurity_decrease': 0.0, 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'criterion': 'mse', 'random_state': None, 'min_impurity_split': None, 'max_features': None, 'max_depth': 10}


In [204]:
y_pred = dt_regressor.predict(X_test)
r2 = r2_score_metric(y_test, y_pred)
print('optimal model has R^2 score {:,.2f} on test data'.format(r2))

optimal model has R^2 score 0.65 on test data


In [1]:
# print features_encoded[0:2]
# result = dt_regressor.predict(features_encoded)

# print result
# result = {"h_price_pred": result[:,0],
#           "l_price_pred": result[:,1]}
# print result
# result = pd.DataFrame(result)
# print result
# data1 = data
# #print data1
# #data1 = data1.reset_index(drop=True)
# data1.head()
# result = data.join(result)
# result.head()
# #result.to_csv("out_chengdu1.csv", sep=',', index=True, encoding='utf-8')

              0          1
0      8.695455   6.904545
1      8.800000   7.800000
2      4.386486   3.553514
3      2.767240   2.168909
4     10.000000   8.000000
5      2.767240   2.168909
6      6.770000   5.527500
7      3.997514   2.769199
8      3.997514   2.769199
9      3.997514   2.769199
10     3.997514   2.769199
11     2.767240   2.168909
12    12.500000  10.500000
13    13.000000  11.000000
14     3.997514   2.769199
15     2.767240   2.168909
16     5.393103   4.001724
17     5.143103   4.237069
18     2.767240   2.168909
19     2.767240   2.168909
20     2.767240   2.168909
21     2.767240   2.168909
22     2.767240   2.168909
23     8.695455   6.904545
24     8.800000   7.800000
25     4.386486   3.553514
26     2.767240   2.168909
27    10.000000   8.000000
28     2.767240   2.168909
29     6.770000   5.527500
...         ...        ...
7341   3.997514   2.769199
7342   3.997514   2.769199
7343   3.997514   2.769199
7344  14.000000   9.875000
7345  10.000000   8.500000
7

TypeError: unhashable type

## 3. 结论